In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import math

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from collections import deque

import cv2
import operator

sys.path.append("..")

from utils import label_map_util

from utils import visualization_utils as vis_util

#MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
#MODEL_NAME = 'indoor_training_dir'
MODEL_NAME = 'Indoor_inference_graph'

#PATH_TO_LABELS = os.path.join('data', 'door_label_map.pbtxt')
PATH_TO_LABELS = os.path.join('data', 'indoor_labelmap.pbtxt')
NUM_CLASSES = 1

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [ ]:
def getCoodinatesFound(image,boxes,scores):
    det_coords = []
    im_height, im_width = image_np.shape[:2]
    max_boxes_to_draw = boxes[0].shape[0]
    min_score_thresh=.5
    for i in range(min(max_boxes_to_draw, boxes[0].shape[0])):
        if scores is None or scores[0][i] > min_score_thresh:
            class_name = category_index[classes[0][i]]['name']
            position = boxes[0][i]
            (xmin, xmax, ymin, ymax) = (position[1]*im_width, position[3]*im_width, position[0]*im_height, position[2]*im_height)
            det_coords.append([int(xmin),int(xmax),int(ymin),int(ymax)])
    
    s = sorted(det_coords,key = operator.itemgetter(0))
    return s

In [ ]:
def get_crosspt(x11,y11, x12,y12, x21,y21, x22,y22):
    if x12==x11 or x22==x21:
        print('delta x=0')
        if x12==x11:
            cx = x12
            m2 = (y22 - y21) / (x22 - x21)
            cy = m2 * (cx - x21) + y21
            return cx, cy
        if x22==x21:
            cx = x22
            m1 = (y12 - y11) / (x12 - x11)
            cy = m1 * (cx - x11) + y11
            return cx, cy

    m1 = (y12 - y11) / (x12 - x11)
    m2 = (y22 - y21) / (x22 - x21)
    if m1==m2:
        print('parallel')
        return None
    cx = (x11 * m1 - y11 - x21 * m2 + y21) / (m1 - m2)
    cy = m1 * (cx - x11) + y11

    return cx, cy

In [ ]:
def EuclideanD(f1,f2):
    x1 = f1[0]
    y1 = f1[1]
    x2 = f2[0]
    y2 = f2[1]
    dx = x2-x1
    dy = y2-y1
    d_len = math.sqrt((dx*dx)+(dy*dy))
    return d_len

In [ ]:
inputpath = '1input/indoor/'
s = '1124_1'
inputimage = inputpath+s+'.jpg'
print("Input image: "+ inputimage+'\n')
image_np = cv2.imread(inputimage, cv2.IMREAD_COLOR)

print(s)
print('\n')

im_height, im_width, im_channels = image_np.shape
font = cv2.FONT_HERSHEY_SIMPLEX 
fontScale = 7
color = (255, 0, 0)   
thickness = 2

#coords_found=[]
lines_above_90 = []
lines_below_90 = []

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
           
        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
          
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')      
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        (boxes, scores, classes, num_detections) = sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        vis_util.visualize_boxes_and_labels_on_image_array(
                 image_np,
                 np.squeeze(boxes),
                 np.squeeze(classes).astype(np.int32),
                 np.squeeze(scores),
                 category_index, 
                 use_normalized_coordinates=True,
                 line_thickness=8)
#                 min_score_thresh=0.8)     
        
        coords_found = getCoodinatesFound(image_np,boxes,scores)

for coord in coords_found:
    cv2.rectangle(image_np,(coord[0],coord[2]),(coord[1],coord[3]),(0,255,0),8)

cv2.imwrite('1output/indoor/test/bothside/'+s+'/detect.jpg',image_np)

for i in range(0,len(coords_found)):
    coords_found[i].append(coords_found[i][1]-coords_found[i][0])

coords_found = sorted(coords_found,key = operator.itemgetter(4))    
for i in range(0,len(coords_found)):
    del coords_found[i][4]


print("coords found: "+str(coords_found) + "  #xmin,xmax,ymin,ymax")
print('\n')

##########################first coords found####################################

xmin,xmax,ymin,ymax = coords_found[len(coords_found)-1]

crop_img = image_np[ymin:ymax, xmin:xmax]
cv2.imwrite('1output/indoor/test/bothside/'+s+'/cropimg1.jpg',crop_img)

gray = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel,iterations=2)
sure_bg = cv2.dilate(opening,kernel,iterations=3)
edges = cv2.Canny(sure_bg,100,140,apertureSize = 3)
#cv2.imwrite('1output/indoor/test/bothside/edge1_1.jpg',edges)
edges2 = cv2.dilate(edges,kernel,iterations=3)
#cv2.imwrite('1output/indoor/test/bothside/edge1_2.jpg',edges2)
threshold = 100  
minLineLength = 200  
maxLineGap = 100
lines =cv2.HoughLinesP(edges2,1,np.pi/180,100,minLineLength,maxLineGap)
a,b,c = lines.shape
lines_coords=[]
for line in lines:
    for x1,y1,x2,y2 in line:
        dx = x2-x1
        dy = y2-y1
        d_len = math.sqrt((dx*dx)+(dy*dy))
        if y1>y2:
            degree = abs((np.arctan2(y1-y2,x2-x1)*180)/np.pi)
        elif y1<y2:
            degree = 180 - abs((np.arctan2(y2-y1,x2-x1)*180)/np.pi)
                                    
        if(10<degree<80 or 105<degree<170):
#             cv2.line(crop_img,(x1,y1),(x2,y2),(0,255,0),8)
            lines_coords.append([d_len,x1,x2,y1,y2,degree]) 
cv2.imwrite('1output/indoor/test/bothside/'+s+'/hough1.jpg',crop_img)

###################################### first line sort ####################################

sorted_line = sorted(lines_coords,key = operator.itemgetter(5))
if len(sorted_line)==1:
    line1_1 = sorted_line[0]
    line1_1[1] = line1_1[1]+xmin
    line1_1[2] = line1_1[2]+xmin
    line1_1[3] = line1_1[3]+ymin
    line1_1[4] = line1_1[4]+ymin
    if line1_1[5] > 90:
        lines_above_90.append(line1_1)

    elif line1_1[5] < 90:
        lines_below_90.append(line1_1)
    
elif len(sorted_line)>=2:
    line1_1 = sorted_line[0]
    line1_2 = sorted_line[len(sorted_line)-1]
    line1_1[1] = line1_1[1]+xmin
    line1_1[2] = line1_1[2]+xmin
    line1_1[3] = line1_1[3]+ymin
    line1_1[4] = line1_1[4]+ymin
    line1_2[1] = line1_2[1]+xmin
    line1_2[2] = line1_2[2]+xmin
    line1_2[3] = line1_2[3]+ymin
    line1_2[4] = line1_2[4]+ymin
    if line1_1[5] > 90:
        lines_above_90.append(line1_1)
        if line1_2[5] < 90:
            lines_below_90.append(line1_2)
    elif line1_1[5] < 90:
        lines_below_90.append(line1_1)
        if line1_2[5] > 90:
            lines_above_90.append(line1_2)

print("###first iteration###")
print("lines above 90': "+  str( lines_above_90))
print("lines below 90': "+  str( lines_below_90) + '\n')

##########################second coords found####################################

xmin2,xmax2,ymin2,ymax2 = coords_found[len(coords_found)-2]
crop_img2 = image_np[ymin2:ymax2, xmin2:xmax2]
cv2.imwrite('1output/indoor/test/bothside/'+s+'/cropimg2.jpg',crop_img2)            

gray2 = cv2.cvtColor(crop_img2,cv2.COLOR_BGR2GRAY)
ret2, thresh2 = cv2.threshold(gray2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
opening2 = cv2.morphologyEx(thresh2,cv2.MORPH_OPEN,kernel,iterations=2)
sure_bg2 = cv2.dilate(opening2,kernel,iterations=3)
edges3 = cv2.Canny(sure_bg2,100,140,apertureSize = 3)
#cv2.imwrite('1output/indoor/test/bothside/edge2_1.jpg',edges3)
edges4 = cv2.dilate(edges3,kernel,iterations=3)
cv2.imwrite('1output/indoor/test/bothside/'+s+'/edge2_2.jpg',edges4)

threshold = 100  
minLineLength = 200  
maxLineGap = 100  
lines2 =cv2.HoughLinesP(edges4,1,np.pi/180,100,minLineLength,maxLineGap)
a2,b2,c2 = lines2.shape
lines_coords2=[]
for line in lines2:
    for x1,y1,x2,y2 in line:
        dx = x2-x1
        dy = y2-y1
        d_len = math.sqrt((dx*dx)+(dy*dy))
        if y1>y2:
            degree = abs((np.arctan2(y1-y2,x2-x1)*180)/np.pi)
        elif y1<y2:
            degree = 180 - abs((np.arctan2(y2-y1,x2-x1)*180)/np.pi)
                                    
        if(10<degree<80 or 105<degree<170):
#             cv2.line(crop_img2,(x1,y1),(x2,y2),(0,255,0),8)
            lines_coords2.append([d_len,x1,x2,y1,y2,degree]) 
cv2.imwrite('1output/indoor/test/bothside/'+s+'/hough2.jpg',crop_img2)

###################################### second line sort ####################################

sorted_line2 = sorted(lines_coords2,key = operator.itemgetter(5))

if len(sorted_line2)==1:
    line2_1 = sorted_line2[0]
    line2_1[1] = line2_1[1]+xmin2
    line2_1[2] = line2_1[2]+xmin2
    line2_1[3] = line2_1[3]+ymin2
    line2_1[4] = line2_1[4]+ymin2
    if line2_1[5] > 90:
        lines_above_90.append(line2_1)
    elif line2_1[5] < 90:
        lines_below_90.append(line2_1)
    
elif len(sorted_line2)>=2:
    line2_1 = sorted_line2[0]
    line2_2 = sorted_line2[len(sorted_line2)-1]
    line2_1[1] = line2_1[1]+xmin2
    line2_1[2] = line2_1[2]+xmin2
    line2_1[3] = line2_1[3]+ymin2
    line2_1[4] = line2_1[4]+ymin2
    line2_2[1] = line2_2[1]+xmin2
    line2_2[2] = line2_2[2]+xmin2
    line2_2[3] = line2_2[3]+ymin2
    line2_2[4] = line2_2[4]+ymin2
    if line2_1[5] > 90:
        lines_above_90.append(line2_1)
        if line2_2[5] < 90:
            lines_below_90.append(line2_2)
    elif line2_1[5] < 90:
        lines_below_90.append(line2_1)
        if line2_2[5] > 90:
            lines_above_90.append(line2_2)

print("###second iteration###")            
print("lines above 90': "+  str( lines_above_90))
print("lines below 90': "+  str( lines_below_90) + '\n')        

##############################line selection##################################
if len(lines_above_90)==0:
    print("no line detected\n")

line_1 = lines_above_90[0]
if len(lines_below_90)>1:
    x_dif1 = abs(lines_below_90[0][1] -line_1[1])
    x_dif2 = abs(lines_below_90[1][1] -line_1[1])
    if x_dif1 > x_dif2:
        line_2 = lines_below_90[0]
    else:
        line_2 = lines_below_90[1]
else:
    line_2 = lines_below_90[0]
        
print("#d_len,x1,x2,y1,y2,degree")
print("line1: " + str(line_1))
print("line2: " + str(line_2))
print('\n')
cv2.line(image_np,(line_1[1],line_1[3]),(line_1[2],line_1[4]),(0,0,255),8)
cv2.line(image_np,(line_2[1],line_2[3]),(line_2[2],line_2[4]),(0,0,255),8)
cv2.imwrite('1output/indoor/test/bothside/1132/wall.jpg',image_np)   

vanishing_pt = get_crosspt(line_1[1],line_1[3],line_1[2],line_1[4],
                                      line_2[1],line_2[3],line_2[2],line_2[4])
print("Vanishing Point: "+str(vanishing_pt)+'\n')
cv2.line(image_np,(int(vanishing_pt[0]),int(vanishing_pt[1])),(int(vanishing_pt[0]),int(vanishing_pt[1])),(0,0,0),20)
        
left_coords=[]
right_coords=[]
    
for coords in coords_found:
    if coords[0] > vanishing_pt[0]:
        right_coords.append(coords)
    if coords[0] < vanishing_pt[0]:        
        left_coords.append(coords)
        
left_coords = sorted(left_coords,key = operator.itemgetter(0))  
right_coords = sorted(right_coords,key = operator.itemgetter(0)) 
print("left coords: "+str(left_coords))
print("right coords: "+str(right_coords)+'\n')        


##################################2D factor###################################

right_factor_2D = []
left_factor_2D = []
if len(left_coords) == 0: ##################################right side <2D factor>
    print("#####right side##### \n")
    side="right"
    for i in range(0,len(right_coords)):
        x1 = right_coords[i][0]
        x2 = right_coords[i][1]
        y1 = (line_1[4]-line_1[3])/(line_1[2]-line_1[1])*(x1-line_1[1])+line_1[3]
        y2 = (line_1[4]-line_1[3])/(line_1[2]-line_1[1])*(x2-line_1[1])+line_1[3]
        right_factor_2D.append([x1,y1])
        right_factor_2D.append([x2,y2])
        
    print("right 2d factor: "+ str(right_factor_2D)+'\n')
    
    for i in range(0,len(right_factor_2D)):
        cv2.putText(image_np, '2Dfactor'+str(i), (int(right_factor_2D[i][0]),int(right_factor_2D[i][1])), font,  
            2, color, thickness, cv2.LINE_AA)  
        cv2.line(image_np,(int(right_factor_2D[i][0]),int(right_factor_2D[i][1])),
            (int(right_factor_2D[i][0]),int(right_factor_2D[i][1])),color,30)

#     cv2.imwrite('1output/indoor/test/bothside/801/intersection.jpg',image_np)        

elif len(right_coords) == 0: ##################################left side <2D factor>
    print("#####left side##### \n")
    side="left"
    for i in range(0,len(left_coords)):
        x1 = left_coords[i][0]
        x2 = left_coords[i][1]
        y1 = (line_2[4]-line_2[3])/(line_2[2]-line_2[1])*(x1-line_2[1])+line_2[3]
        y2 = (line_2[4]-line_2[3])/(line_2[2]-line_2[1])*(x2-line_2[1])+line_2[3]
        left_factor_2D.append([x1,y1])
        left_factor_2D.append([x2,y2])    
        
    print("left 2d factor: "+ str(left_factor_2D))                   
    
    for i in range(0,len(left_factor_2D)):
        cv2.putText(image_np, '2Dfactor'+str(i), (int(left_factor_2D[i][0]),int(left_factor_2D[i][1])), font,  
            2, color, thickness, cv2.LINE_AA)  
        cv2.line(image_np,(int(left_factor_2D[i][0]),int(left_factor_2D[i][1])),
            (int(left_factor_2D[i][0]),int(left_factor_2D[i][1])),color,30)
    

else: ##################################both side <2D factor>
    print("#####both side##### \n")
    side="both"
    if line_1[1] > vanishing_pt[0]:
        for i in range(0,len(right_coords)):
            x1 = right_coords[i][0]
            x2 = right_coords[i][1]
            y1 = (line_1[4]-line_1[3])/(line_1[2]-line_1[1])*(x1-line_1[1])+line_1[3]
            y2 = (line_1[4]-line_1[3])/(line_1[2]-line_1[1])*(x2-line_1[1])+line_1[3]
            right_factor_2D.append([x1,y1])
            right_factor_2D.append([x2,y2]) 
    
        for i in range(0,len(left_coords)):
            x1 = left_coords[i][0]
            x2 = left_coords[i][1]
            y1 = (line_2[4]-line_2[3])/(line_2[2]-line_2[1])*(x1-line_2[1])+line_2[3]
            y2 = (line_2[4]-line_2[3])/(line_2[2]-line_2[1])*(x2-line_2[1])+line_2[3]
            left_factor_2D.append([x1,y1])
            left_factor_2D.append([x2,y2]) 
    
    else:
        for i in range(0,len(left_coords)):
            x1 = left_coords[i][0]
            x2 = left_coords[i][1]
            y1 = (line_1[4]-line_1[3])/(line_1[2]-line_1[1])*(x1-line_1[1])+line_1[3]
            y2 = (line_1[4]-line_1[3])/(line_1[2]-line_1[1])*(x2-line_1[1])+line_1[3]
            left_factor_2D.append([x1,y1])
            left_factor_2D.append([x2,y2]) 
    
        for i in range(0,len(right_coords)):
            x1 = right_coords[i][0]
            x2 = right_coords[i][1]
            y1 = (line_2[4]-line_2[3])/(line_2[2]-line_2[1])*(x1-line_2[1])+line_2[3]
            y2 = (line_2[4]-line_2[3])/(line_2[2]-line_2[1])*(x2-line_2[1])+line_2[3]
            right_factor_2D.append([x1,y1])
            right_factor_2D.append([x2,y2])     

    print("left 2d factor: "+ str(left_factor_2D))                   
    print("right 2d factor: "+ str(right_factor_2D)+'\n')
    
    for i in range(0,len(left_factor_2D)):
        cv2.putText(image_np, '2Dfactor'+str(i)+'(left)', (int(left_factor_2D[i][0]),int(left_factor_2D[i][1])), font,  
            2, color, thickness, cv2.LINE_AA)  
        cv2.line(image_np,(int(left_factor_2D[i][0]),int(left_factor_2D[i][1])),
            (int(left_factor_2D[i][0]),int(left_factor_2D[i][1])),color,30)

    for i in range(0,len(right_factor_2D)):
        cv2.putText(image_np, '2Dfactor'+str(i)+'(right)', (int(right_factor_2D[i][0]),int(right_factor_2D[i][1])), font,  
            2, color, thickness, cv2.LINE_AA)  
        cv2.line(image_np,(int(right_factor_2D[i][0]),int(right_factor_2D[i][1])),
            (int(right_factor_2D[i][0]),int(right_factor_2D[i][1])),color,30)


##################################Real Distance Ratio################################        
print("-----Real Distance Ratio--------\n")
if side == "left":
    left_real_ratio = [1]
    for i in range(0,int(len(left_factor_2D))-2):
        next_ratio = 1/((( EuclideanD(left_factor_2D[i],left_factor_2D[i+2]) * EuclideanD(left_factor_2D[i+1],vanishing_pt) )/
        ( EuclideanD(left_factor_2D[i+1],left_factor_2D[i+2]) * EuclideanD(left_factor_2D[i],vanishing_pt) ))-1) * left_real_ratio[i]                   
        left_real_ratio.append(abs(next_ratio))
        
    print("left ratio: " + str(left_real_ratio)+'\n')

if side == "right":
    right_factor_2D.reverse()
    right_real_ratio = [1]
    for i in range(0,int(len(right_factor_2D))-2):
        next_ratio = 1/((( EuclideanD(right_factor_2D[i],right_factor_2D[i+2]) * EuclideanD(right_factor_2D[i+1],vanishing_pt) )/
        ( EuclideanD(right_factor_2D[i+1],right_factor_2D[i+2]) * EuclideanD(right_factor_2D[i],vanishing_pt) ))-1) * right_real_ratio[i]                   
        right_real_ratio.append(abs(next_ratio))
        
    print("right ratio: " + str(right_real_ratio)+'\n')

elif side == "both":         ############################both side #############################
    if line_1[1] > vanishing_pt[0]: # line1 == right, line2 == left
        b_left= line_2[3] - line_2[1]*(line_2[4]-line_2[3])/(line_2[2]-line_2[1])
        b_right = line_1[3] + (im_width-line_1[1])*(line_1[4]-line_1[3])/(line_1[2]-line_1[1])
    elif line_2[1] > vanishing_pt[0]: # line1 == left, line2 == right 
        b_left= line_1[3] - line_1[1]*(line_1[4]-line_1[3])/(line_1[2]-line_1[1])
        b_right = line_2[3] + (im_width-line_2[1])*(line_2[4]-line_2[3])/(line_2[2]-line_2[1])        
    
    cv2.line(image_np,(0,int(b_left)),(0,int(b_left)),(0,0,0),20)
    cv2.line(image_np,(im_width,int(b_right)),(im_width,int(b_right)),(0,0,0),20)
    
    left_start_ratio = ( ( EuclideanD((0,int(b_left)),left_factor_2D[1] ) * EuclideanD(left_factor_2D[0],vanishing_pt) ) 
                        /  ( EuclideanD(left_factor_2D[0],left_factor_2D[1]) * EuclideanD((0,int(b_left)),vanishing_pt) ) ) -1
    
    left_real_ratio = [1]
    for i in range(0,int(len(left_factor_2D))-2):
        next_ratio = 1/((( EuclideanD(left_factor_2D[i],left_factor_2D[i+2]) * EuclideanD(left_factor_2D[i+1],vanishing_pt) )/
        ( EuclideanD(left_factor_2D[i+1],left_factor_2D[i+2]) * EuclideanD(left_factor_2D[i],vanishing_pt) ))-1) * left_real_ratio[i]                   
        left_real_ratio.append(abs(next_ratio)) 
    
    right_factor_2D.reverse()

    right_start_ratio = ( ( EuclideanD((im_width,int(b_right)),right_factor_2D[1] ) * EuclideanD(right_factor_2D[0],vanishing_pt) ) 
                        /  ( EuclideanD(right_factor_2D[0],right_factor_2D[1]) * EuclideanD((im_width,int(b_right)),vanishing_pt) ) ) -1    
    
    right_real_ratio = [1]
    for i in range(0,int(len(right_factor_2D))-2):
        next_ratio = 1/((( EuclideanD(right_factor_2D[i],right_factor_2D[i+2]) * EuclideanD(right_factor_2D[i+1],vanishing_pt) )/
        ( EuclideanD(right_factor_2D[i+1],right_factor_2D[i+2]) * EuclideanD(right_factor_2D[i],vanishing_pt) ))-1) * right_real_ratio[i]                   
        right_real_ratio.append(abs(next_ratio))
    print("left start ratio: "+str(left_start_ratio))
    print("left ratio: " + str(left_real_ratio)+'\n')
    print("right start ratio: "+str(right_start_ratio))
    print("right ratio: " + str(right_real_ratio)+'\n')

print("---------------------------------\n")


#######################################sketch#############################################    
sketch = cv2.imread("indoor_input/sketch.png", cv2.IMREAD_COLOR)    
sk_height, sk_width = sketch.shape[:2]
print("sketch height: "+str(sk_height))
print("sketch width: "+str(sk_width))
print("\n---------- blueprint ------------\n")
if side =="left":
    left_ratio_sum = 0
    for i in range(0,len(left_real_ratio)):
        left_ratio_sum = left_ratio_sum + left_real_ratio[i]

    sketch_ratio = []
    for i in range(0,len(left_real_ratio)):
        sketch_ratio.append(sk_height*(left_real_ratio[i]/left_ratio_sum))
    
    h_flag = 0
    sketch_ratio.reverse()
    a = 0
    b = 0
    for i in range(0,len(sketch_ratio)):
        if i%2==0:
            b = b + int(sketch_ratio[i])
            print("door(left): "+str(a))
            print("door(right): "+str(b)+'\n')
            cv2.line(sketch,(0,a),(0,b),(255,0,0),5)
            a=b
    
        else:
            a = a + int(sketch_ratio[i])
            b = b + int(sketch_ratio[i])        

            
elif side =="right":
    right_ratio_sum = 0
    for i in range(0,len(right_real_ratio)):
        right_ratio_sum = right_ratio_sum + right_real_ratio[i]

    sketch_ratio = []
    for i in range(0,len(right_real_ratio)):
        sketch_ratio.append(sk_height*(right_real_ratio[i]/right_ratio_sum))
    
    h_flag = 0
    sketch_ratio.reverse()
    a = 0
    b = 0
    for i in range(0,len(sketch_ratio)):
        if i%2==0:
            b = b + int(sketch_ratio[i])
            print("door(left): "+str(a))
            print("door(right): "+str(b)+'\n')
            cv2.line(sketch,(sk_width,a),(sk_width,b),(255,0,0),5)
            a=b
    
        else:
            a = a + int(sketch_ratio[i])
            b = b + int(sketch_ratio[i])
            
            
elif side =="both": 
    if len(left_real_ratio) > len(right_real_ratio):
        left_ratio_sum = 0
        for i in range(0,len(left_real_ratio)):
            left_ratio_sum = left_ratio_sum + left_real_ratio[i]
        left_ratio_sum = left_ratio_sum + left_start_ratio
    
        left_sketch_ratio = [sk_height*(left_start_ratio/left_ratio_sum)]
        for i in range(0,len(left_real_ratio)):
            left_sketch_ratio.append(sk_height*(left_real_ratio[i]/left_ratio_sum))

        left_h_flag = 0
        left_sketch_ratio.reverse()
        a = 0
        b = 0
        for i in range(0,len(left_sketch_ratio)):
            if i%2==0:
                b = b + int(left_sketch_ratio[i])
                print("door(left): "+str(a))
                print("door(right): "+str(b)+'\n')
                cv2.line(sketch,(0,a),(0,b),(255,0,0),5)
                a=b
    
            else:
                b = b + int(left_sketch_ratio[i])
                cv2.line(sketch,(0,a),(0,b),(0,0,255),5)
                a = a + int(left_sketch_ratio[i])
                      
        left_start_sketch_ratio = int(sk_height*(left_start_ratio/left_ratio_sum))
        cv2.line(sketch,(0,b),(0,b+left_start_sketch_ratio),(0,0,255),5) 
    

        right_sketch_ratio = []
        for i in range(0,len(right_real_ratio)):
            right_sketch_ratio.append(sk_height*(right_real_ratio[i]/left_ratio_sum))
    
        right_h_flag = 0
        right_sketch_ratio.reverse()
        a = 0
        b = 0
        for i in range(0,len(right_sketch_ratio)):
            if i%2==0:
                b = b + int(right_sketch_ratio[i])
                print("door(left): "+str(a))
                print("door(right): "+str(b)+'\n')
                cv2.line(sketch,(sk_width,a),(sk_width,b),(255,0,0),5)
                a=b
    
            else:
                b = b + int(right_sketch_ratio[i])
                cv2.line(sketch,(sk_width,a),(sk_width,b),(0,0,255),5)            
                a = a + int(right_sketch_ratio[i])
    
        right_start_sketch_ratio = int(sk_height*(right_start_ratio/left_ratio_sum))
        cv2.line(sketch,(sk_width,b),(sk_width,b+right_start_sketch_ratio),(0,0,255),5)         
    
    
    elif len(left_real_ratio) < len(right_real_ratio):
        right_ratio_sum = 0
        for i in range(0,len(right_real_ratio)):
            right_ratio_sum = right_ratio_sum + right_real_ratio[i]
        right_ratio_sum = right_ratio_sum + right_start_ratio

        right_sketch_ratio = []
        for i in range(0,len(right_real_ratio)):
            right_sketch_ratio.append(sk_height*(right_real_ratio[i]/right_ratio_sum))
    
        right_h_flag = 0
        right_sketch_ratio.reverse()
        a = 0
        b = 0
        for i in range(0,len(right_sketch_ratio)):
            if i%2==0:
                b = b + int(right_sketch_ratio[i])
                print("door(left): "+str(a))
                print("door(right): "+str(b)+'\n')
                cv2.line(sketch,(sk_width,a),(sk_width,b),(255,0,0),5)
                a=b
    
            else:
                b = b + int(right_sketch_ratio[i])
                cv2.line(sketch,(sk_width,a),(sk_width,b),(0,0,255),5)            
                a = a + int(right_sketch_ratio[i])
    
        right_start_sketch_ratio = int(sk_height*(right_start_ratio/right_ratio_sum))
        cv2.line(sketch,(sk_width,b),(sk_width,b+right_start_sketch_ratio),(0,0,255),5)         
        
        
        left_ratio_sum = 0
        for i in range(0,len(left_real_ratio)):
            left_ratio_sum = left_ratio_sum + left_real_ratio[i]
        left_ratio_sum = left_ratio_sum + left_start_ratio
    
        left_sketch_ratio = [sk_height*(left_start_ratio/left_ratio_sum)]
        for i in range(0,len(left_real_ratio)):
            left_sketch_ratio.append(sk_height*(left_real_ratio[i]/left_ratio_sum))

        left_h_flag = 0
        left_sketch_ratio.reverse()
        a = 0
        b = 0
        for i in range(0,len(left_sketch_ratio)):
            if i%2==0:
                b = b + int(left_sketch_ratio[i])
                print("door(left): "+str(a))
                print("door(right): "+str(b)+'\n')
                cv2.line(sketch,(0,a),(0,b),(255,0,0),5)
                a=b
    
            else:
                b = b + int(left_sketch_ratio[i])
                cv2.line(sketch,(0,a),(0,b),(0,0,255),5)
                a = a + int(left_sketch_ratio[i])
                      
        left_start_sketch_ratio = int(sk_height*(left_start_ratio/left_ratio_sum))
        cv2.line(sketch,(0,b),(0,b+left_start_sketch_ratio),(0,0,255),5) 

    
    
print("--------------------------------\n")          
# cv2.imwrite('1output/indoor/test/bothside/blueprint.jpg',sketch)   
# cv2.imwrite('1output/indoor/test/bothside/result.jpg',image_np)
# print("Done")

blueprint_write = '1output/indoor/test/bothside/'+s+'/blueprint'+s+'.jpg'
result_write = '1output/indoor/test/bothside/'+s+'/result'+s+'.jpg'

print("Done")
cv2.imwrite(blueprint_write,sketch)        
cv2.imwrite(result_write,image_np)

In [ ]:
class Node:
    def __init__(self, val, name,length,pixel):
        self.val = val
        self.name = name
        self.length = length
        self.pixel=pixel
        self.edges = []


class Graph:
    def __init__(self):
        self.nodes = []

    def add_node(self, val, name, length,pixel):
        new_node = Node(val,name,length,pixel)
        self.nodes.append(new_node)

    def add_edge(self, node1, node2):
        self.nodes[node1].edges.append(self.nodes[node2])
        self.nodes[node2].edges.append(self.nodes[node1])

    def dfs(self):
        if not self.nodes:
            return []
        start = self.nodes[0] 
        visited, stack, result = set([start]), [start], []

        while stack:
            node = stack.pop()
            result.append(node)
            for nd in node.edges:
                if nd not in visited:
                    stack.append(nd)
                    visited.add(nd)
                    
        return result

     
right_graph = Graph()
right_graph.add_node(0,'root',-1,(0,0))
right_graph.add_node(1,'wall',3,(0,0))
right_graph.add_node(2,'door',1,(0,0))

right_graph.add_node(3,'wall',1.5,(0,0))
right_graph.add_node(4,'door',1,(0,0))
right_graph.add_node(5,'wall',1.45,(0,0))
right_graph.add_node(6,'door',1,(0,0))

right_graph.add_node(7,'wall',3.4,(0,0))
right_graph.add_node(8,'door',1,(375,200))
right_graph.add_node(9,'wall',0.5,(0,0))
right_graph.add_node(10,'door',1,(430,210))
right_graph.add_node(11,'wall',2.26,(0,0))
right_graph.add_node(12,'door',1,(544,210))
right_graph.add_node(13,'wall',2.26,(0,0))
right_graph.add_node(14,'door',1,(661,210))
right_graph.add_node(15,'wall',2.26,(0,0))
right_graph.add_node(16,'door',1,(777,210))


right_graph.add_edge(0,1)
right_graph.add_edge(0,3)
right_graph.add_edge(0,7)

right_graph.add_edge(1,2)

right_graph.add_edge(3,4)
right_graph.add_edge(4,5)
right_graph.add_edge(5,6)

right_graph.add_edge(7,8)
right_graph.add_edge(8,9)
right_graph.add_edge(9,10)
right_graph.add_edge(10,11)
right_graph.add_edge(11,12)
right_graph.add_edge(12,13)
right_graph.add_edge(13,14)
right_graph.add_edge(14,15)
right_graph.add_edge(15,16)


#                      2
#                     /
#                    1
#                   /
#  6 - 5 - 4 - 3 - 0 -  7 - 8 - 9 ~ 16


left_graph = Graph()
left_graph.add_node(100,'root',-1,(0,0))

left_graph.add_node(101,'door',1,(0,0))
left_graph.add_node(102,'wall',1.4,(0,0))
left_graph.add_node(103,'door',1,(0,0))

left_graph.add_node(104,'door',1,(0,0))
left_graph.add_node(105,'wall',2,(0,0))
left_graph.add_node(106,'door',1,(0,0))

left_graph.add_node(107,'door',1,(0,0))
left_graph.add_node(108,'wall',2.8,(0,0))
left_graph.add_node(109,'door',1.8,(0,0))
left_graph.add_node(110,'wall',4.65,(0,0))
left_graph.add_node(111,'door',1.8,(0,0))

left_graph.add_edge(0,1)
left_graph.add_edge(0,4)
left_graph.add_edge(0,7)

left_graph.add_edge(1,2)
left_graph.add_edge(2,3)

left_graph.add_edge(4,5)
left_graph.add_edge(5,6)

left_graph.add_edge(7,8)
left_graph.add_edge(8,9)
left_graph.add_edge(9,10)
left_graph.add_edge(10,11)


#                             103
#                             /
#                           102
#                           /
#                         101
#                         /
#      106 - 105 - 104 - 100 -  107 - 108 - 109 - 110 - 111


In [ ]:
print("-----------Right Graph-----------")      #####RIGHT SIDE
right_start_nodes = right_graph.nodes[0].edges
right_end_nodes=[]
right_hall_nodes = []
right_hall_nodes_reverse = []
for i in range(len(right_start_nodes)+1):
    right_hall_nodes.append([])
    right_hall_nodes_reverse.append([])
    
right_dfs_result = right_graph.dfs()
edge_flag = 0
for i in range(len(right_dfs_result)): 
    for n in right_start_nodes:
        if right_dfs_result[i].val == n.val:
            edge_flag=edge_flag+1
    right_hall_nodes[edge_flag].append(right_dfs_result[i])        

#hall nodes - forward
print("-----hall nodes - Forward-----")
for i in range(len(right_hall_nodes)):
    right_end_nodes.append(right_hall_nodes[i][len(right_hall_nodes[i])-1])
    right_hall_nodes_reverse[i]= list(reversed(right_hall_nodes[i]))
    print("------")
    for j in range(len(right_hall_nodes[i])):
        print(right_hall_nodes[i][j].val, right_hall_nodes[i][j].name, 
              right_hall_nodes[i][j].length, right_hall_nodes[i][j].pixel)

#hall nodes - opposite
print("------")
print("-----hall nodes - Backward-----")
for i in range(len(right_hall_nodes_reverse)):
    print("------")
    for j in range(len(right_hall_nodes_reverse[i])):
        print(right_hall_nodes_reverse[i][j].val, right_hall_nodes_reverse[i][j].name, 
              right_hall_nodes_reverse[i][j].length, right_hall_nodes_reverse[i][j].pixel)
        
        
        
print("\n-----------Left Graph-----------")      #####Left SIDE
left_start_nodes = left_graph.nodes[0].edges
left_end_nodes=[]
left_hall_nodes = []
left_hall_nodes_reverse = []
for i in range(len(left_start_nodes)+1):
    left_hall_nodes.append([])
    left_hall_nodes_reverse.append([])
    
left_dfs_result = left_graph.dfs()
edge_flag = 0
for i in range(len(left_dfs_result)): 
    for n in left_start_nodes:
        if left_dfs_result[i].val == n.val:
            edge_flag=edge_flag+1
    left_hall_nodes[edge_flag].append(left_dfs_result[i])        

#hall nodes - forward
print("-----hall nodes - Forward-----")
for i in range(len(left_hall_nodes)):
    left_end_nodes.append(left_hall_nodes[i][len(left_hall_nodes[i])-1])
    left_hall_nodes_reverse[i]= list(reversed(left_hall_nodes[i]))
    print("------")
    for j in range(len(left_hall_nodes[i])):
        print(left_hall_nodes[i][j].val, left_hall_nodes[i][j].name, 
              left_hall_nodes[i][j].length, left_hall_nodes[i][j].pixel)

#hall nodes - opposite
print("------")
print("-----hall nodes - Backward-----")
for i in range(len(left_hall_nodes_reverse)):
    print("------")
    for j in range(len(left_hall_nodes_reverse[i])):
        print(left_hall_nodes_reverse[i][j].val, left_hall_nodes_reverse[i][j].name, 
              left_hall_nodes_reverse[i][j].length, left_hall_nodes_reverse[i][j].pixel)

    

##############Positioning#############

right_ratio1 = [1, 0.5, 0.9, 2.22, 1.25]
#right_ratio1 = [1.8,2.8,1]
estimate_len = len(right_ratio1)


if side == 'right':

    right_position=[]
    for i in range(1,len(right_hall_nodes)): 
        if len(right_hall_nodes[i])>= estimate_len:
            for j in range(0,len(right_hall_nodes[i]) - len(right_ratio1)+1):
                if right_hall_nodes[i][j].name == 'door':
                    real_sum=0
                    for k in range(0,len(right_ratio1)):
                        s=min(right_hall_nodes[i][j+k].length,right_ratio1[k]) / max(right_hall_nodes[i][j+k].length,right_ratio1[k])
                        real_sum = real_sum + s
                    
                    right_position.append([real_sum,right_hall_nodes[i][j],'forward'])

    right_position_reverse=[]
    for i in range(1,len(left_hall_nodes_reverse)): 
        if len(left_hall_nodes_reverse[i])>= estimate_len:
            for j in range(0,len(left_hall_nodes_reverse[i]) - len(right_ratio1)+1):
                if left_hall_nodes_reverse[i][j].name == 'door':
                    real_sum=0
                    for k in range(0,len(right_ratio1)):
                        s=min(left_hall_nodes_reverse[i][j+k].length,right_ratio1[k]) / max(left_hall_nodes_reverse[i][j+k].length,right_ratio1[k])
                        real_sum = real_sum + s
                    
                    right_position_reverse.append([real_sum,left_hall_nodes_reverse[i][j],'backward'])    

    right_position_found1 = sorted(right_position,key = operator.itemgetter(0),reverse=True)[0]
    right_position_found2 = sorted(right_position_reverse,key = operator.itemgetter(0),reverse=True)[0]
    if right_position_found1[0] > right_position_found2[0]:
        right_position_found,ori = right_position_found1[1],right_position_found1[2]
    elif right_position_found1[0] < right_position_found2[0]:
        right_position_found,ori = right_position_found2[1],right_position_found2[2]
    
    print("\n---position---")
    print(right_position_found.val,right_position_found.pixel, ori)

##############Position#############


floor_blp = cv2.imread('1input/indoor/8floor.jpg', cv2.IMREAD_COLOR)

if ori == 'forward':
    position_img = cv2.imread('1input/indoor/right11.jpg', cv2.IMREAD_COLOR)
    position_img=cv2.resize(position_img, dsize=(20, 20), interpolation=cv2.INTER_AREA)
#     floor_blp = cv2.arrowedLine(floor_blp, start_point, end_point,  
#                     color, thickness, tipLength = 0.5)  
elif ori == 'backward':
    position_img = cv2.imread('1input/indoor/left11.jpg', cv2.IMREAD_COLOR)
    position_img=cv2.resize(position_img, dsize=(20,20), interpolation=cv2.INTER_AREA)

floor_blp[right_position_found.pixel[1]:right_position_found.pixel[1]+20 , right_position_found.pixel[0]:right_position_found.pixel[0]+20] = position_img

#cv2.circle(floor_blp, right_position_found.pixel, 5, (0,0,255), -1)

cv2.imshow('Window', floor_blp)
cv2.waitKey(0) 
cv2.destroyAllWindows()